# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

### Action Points

1. Create Choice between Take profit and sema move close.
    - If Sema is above slema for long position, then can we cancel take profit step
    - can we use sema angle to decide whether to close profit or not
    - Fix Merge issue

In [2]:
%%time
data = run_engine(data)

Date List Created
Record num : 13087790
format_tick_time : Completed


100%|██████████████████████████████████| 37187/37187 [00:04<00:00, 7641.80it/s]


get_ohlc         : completed
Record num : 36742
get_tick_indicators  : Completed
encode_x_y_split    : Completed
train_model         : Completed
-------------------------------------
Overall Accuracy : 0.5014287658184787
-------------------------------------
              precision    recall  f1-score   support

       green       0.50      0.49      0.49      3630
         red       0.51      0.51      0.51      3719

    accuracy                           0.50      7349
   macro avg       0.50      0.50      0.50      7349
weighted avg       0.50      0.50      0.50      7349

Wall time: 2min 32s


In [3]:
sys.exit()

SystemExit: 

In [16]:
data["predictions"]

array([1, 1, 1, ..., 0, 0, 0])

In [17]:
data["valid_y"]

array([1, 0, 1, ..., 1, 0, 0])

In [18]:
data["y_map"]

{0: 'green', 1: 'red'}

In [ ]:
profile = ProfileReport(data['df_ohlc'])
profile.to_file(output_file="data/output.html")
send_telegram_message(f'Report created')

In [15]:
data['df_ohlc'].iloc[:10,8:19]

,dir,weekday,hour,min,lema,sema,BBand_upper,BBand_middle,BBand_lower,green_candle,red_candle
0,green,0,3,15,1.135856,1.134930,1.135950,1.134930,1.133911,0,1
1,green,0,3,20,1.135797,1.134875,1.135834,1.134860,1.133886,1,0
2,red,0,3,25,1.135743,1.134830,1.135749,1.134804,1.133860,1,0
3,red,0,3,30,1.135687,1.134783,1.135567,1.134730,1.133892,0,1
4,green,0,3,35,1.135632,1.134734,1.135330,1.134650,1.133970,0,1
5,green,0,3,40,1.135593,1.134726,1.135141,1.134604,1.134068,1,0
6,red,0,3,45,1.135567,1.134744,1.135145,1.134605,1.134066,1,0
7,red,0,3,50,1.135542,1.134762,1.135137,1.134603,1.134070,0,1
8,red,0,3,55,1.135514,1.134768,1.135091,1.134591,1.134091,0,1
9,green,0,4,0,1.135476,1.134748,1.135061,1.134576,1.134091,0,1


In [4]:
data['df_ohlc'].tail()

,DateTime_frmt,open,high,low,close,Volume,num_ticks,candle_size,dir,weekday,...,red_candle,lower_touch,middle_touch,high_touch,HT_trendline,ADX,APO,cdl_hammer,cdl_shootingstar,cdl_engulfing
36737,2022-06-30 23:30:00,1.047720,1.047720,1.047590,1.047665,115,68,0.000130,red,3,...,1,0,0,0,1.047814,18.143937,-0.000260,100,0,0
36738,2022-06-30 23:35:00,1.047700,1.047810,1.047510,1.047530,284,181,0.000300,green,3,...,1,0,1,0,1.047773,17.891910,-0.000290,0,0,0
36739,2022-06-30 23:40:00,1.047535,1.047665,1.047495,1.047615,131,117,0.000170,green,3,...,0,0,0,0,1.047740,17.686338,-0.000306,0,0,0
36740,2022-06-30 23:45:00,1.047630,1.047885,1.047630,1.047795,132,109,0.000255,green,3,...,0,0,1,0,1.047715,16.880167,-0.000305,0,0,0
36741,2022-06-30 23:50:00,1.047800,1.047950,1.047780,1.047870,161,110,0.000170,green,3,...,0,0,0,0,1.047701,16.122117,-0.000288,0,0,0


In [ ]:
cols = ['DateTime_frmt', 'open', 'high', 'low', 'close', 'Volume','dir','lema', 'sema','BBand_upper', 'BBand_middle', 'BBand_lower']

x= data['df_ohlc'][data['df_ohlc']['DateTime_frmt'] > '2022-06-01 05:44:00']
x[cols]

In [20]:
# Get Candle Dir --------------------------------------------------------
data['df_ohlc'].loc[data['df_ohlc']['dir'] == 'up', 'up'] = data['df_ohlc']['close']    
data['df_ohlc'].loc[data['df_ohlc']['dir'] == 'down', 'down'] = data['df_ohlc']['close']    

data['df_ohlc']['cdl_hammer'] = data['df_ohlc']['cdl_hammer'].replace({0:np.nan})
data['df_ohlc']['cdl_shootingstar'] = data['df_ohlc']['cdl_shootingstar'].replace({0:np.nan})
data['df_ohlc']['cdl_hammer'] = np.where(data['df_ohlc']['cdl_hammer'] == 100, data['df_ohlc']['close'], data['df_ohlc']['cdl_hammer'])
data['df_ohlc']['cdl_shootingstar'] = np.where(data['df_ohlc']['cdl_shootingstar'] == -100, data['df_ohlc']['close'], data['df_ohlc']['cdl_shootingstar'])

data['df_ohlc']['cdl_engulfing_up'] = np.nan
data['df_ohlc']['cdl_engulfing_down'] = np.nan
data['df_ohlc']['cdl_engulfing_up'] = data['df_ohlc']['cdl_engulfing_up'].replace({0:np.nan})
data['df_ohlc']['cdl_engulfing_down'] = data['df_ohlc']['cdl_engulfing_down'].replace({0:np.nan})
data['df_ohlc']['cdl_engulfing_up'] = np.where(data['df_ohlc']['cdl_engulfing_up'] == 100, data['df_ohlc']['close'], data['df_ohlc']['cdl_engulfing_up'])
data['df_ohlc']['cdl_engulfing_down'] = np.where(data['df_ohlc']['cdl_engulfing_down'] == -100, data['df_ohlc']['close'], data['df_ohlc']['cdl_engulfing_down'])

In [22]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2022-06-2'
data['plot_stop']   = '2022-06-3'
# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
data['things_to_plot'] = ['ema', 'BBands', 'indicators']
plot_graph(data)

In [ ]:
del data['df_ohlc']['up']
del data['df_ohlc']['down']

generate_result_report(data)